In [ ]:
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f"{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

NameError: name 'pd' is not defined

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures

In [ ]:
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f"{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

CompletedProcess(args=['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'], returncode=3)

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f"{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Write the sequence to a temporary FASTA file
    with open('temp_query.fasta', 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', 'temp_query.fasta', '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
    # Parse the BLAST XML output
    blast_record = NCBIXML.read(io.StringIO(result.stdout))
    if blast_record.alignments:
        return blast_record.alignments[0].hsps[0].score
    return 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Parallelize BLAST queries
def process_sequence(fasta):
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score = get_blast_score(sequence)
    return feature, kmer_feature , blast_score

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_sequence, merged_df['FASTA']))

for feature, kmer_feature,blast_score in results: # ,blast_scores kan ook nog toegevoegd worden
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content','blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Optionally, save the model for future use
import joblib
joblib.dump(model, 'random_forest_model.pkl')

# save the blast scores for future use
df.to_csv(features_df, 'blast scores.csv', index=False)

# Clean up temporary files
os.remove('temp_query.fasta')

ValueError: More than one record found in handle

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures
import joblib

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Write the sequence to a temporary FASTA file
    with open('temp_query.fasta', 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', 'temp_query.fasta', '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
    # Parse the BLAST XML output
    blast_record = NCBIXML.read(io.StringIO(result.stdout))
    if blast_record.alignments:
        return blast_record.alignments[0].hsps[0].score
    return 0

# Check if BLAST scores file exists
blast_scores_file = 'blast_scores.csv'
if os.path.exists(blast_scores_file):
    # Load BLAST scores from file
    blast_scores_df = pd.read_csv(blast_scores_file)
    blast_scores = blast_scores_df['blast_score'].tolist()
else:
    # Compute BLAST scores
    features = []
    kmer_features = []
    blast_scores = []

    # Parallelize BLAST queries
    def process_sequence(fasta):
        sequence = str(Seq(fasta))
        feature = extract_features(sequence)
        kmer_feature = get_kmers(sequence)
        blast_score = get_blast_score(sequence)
        return feature, kmer_feature, blast_score

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(process_sequence, df['FASTA']))

    for feature, kmer_feature, blast_score in results:
        features.append(feature)
        kmer_features.append(kmer_feature)
        blast_scores.append(blast_score)

    # Save BLAST scores to file
    blast_scores_df = pd.DataFrame({'blast_score': blast_scores})
    blast_scores_df.to_csv(blast_scores_file, index=False)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# save the blast scores for future use
df.to_csv(features_df, 'blast scores.csv', index=False)

# Clean up temporary files
os.remove('temp_query.fasta')

ValueError: Your XML file was empty

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f"{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Write the sequence to a temporary FASTA file
    with open('temp_query.fasta', 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', 'temp_query.fasta', '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
        # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_record = NCBIXML.read(io.StringIO(result.stdout))
        if blast_record.alignments:
            return blast_record.alignments[0].hsps[0].score
    return 0
# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Parallelize BLAST queries
def process_sequence(fasta):
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score = get_blast_score(sequence)
    return feature, kmer_feature , blast_score

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_sequence, merged_df['FASTA']))

for feature, kmer_feature,blast_score in results: # ,blast_scores kan ook nog toegevoegd worden
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content','blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Optionally, save the model for future use
import joblib
joblib.dump(model, 'random_forest_model.pkl')

# save the blast scores for future use
df.to_csv(features_df, 'blast scores.csv', index=False)

# Clean up temporary files
os.remove('temp_query.fasta')

ValueError: More than one record found in handle

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures
import joblib

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Write the sequence to a temporary FASTA file
    with open('temp_query.fasta', 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', 'temp_query.fasta', '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_record = NCBIXML.read(io.StringIO(result.stdout))
        if blast_record.alignments:
            return blast_record.alignments[0].hsps[0].score
    return 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Parallelize BLAST queries
def process_sequence(fasta):
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score = get_blast_score(sequence)
    return feature, kmer_feature, blast_score

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_sequence, merged_df['FASTA']))

for feature, kmer_feature, blast_score in results:
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Optionally, save the model for future use
joblib.dump(model, 'random_forest_model.pkl')

# Save the blast scores for future use
features_df.to_csv('blast_scores.csv', index=False)

# Clean up temporary files
os.remove('temp_query.fasta')

ValueError: More than one record found in handle

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures
import joblib

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Write the sequence to a temporary FASTA file
    with open('temp_query.fasta', 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', 'temp_query.fasta', '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_record = NCBIXML.read(io.StringIO(result.stdout))
        if blast_record.alignments:
            return blast_record.alignments[0].hsps[0].score
    return 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Parallelize BLAST queries
def process_sequence(fasta):
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score = get_blast_score(sequence)
    return feature, kmer_feature, blast_score

for feature, kmer_feature, blast_score in results:
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Optionally, save the model for future use
joblib.dump(model, 'random_forest_model.pkl')

# Save the blast scores for future use
features_df.to_csv('blast_scores.csv', index=False)

# Clean up temporary files
os.remove('temp_query.fasta')

NameError: name 'results' is not defined

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures
import joblib
import threading

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures
import joblib
import threading

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Use a unique filename for each thread
    temp_fasta_filename = f'temp_query_{threading.get_ident()}.fasta'
    
    # Write the sequence to a temporary FASTA file
    with open('temp_query.fasta', 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', 'temp_query.fasta', '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_record = NCBIXML.read(io.StringIO(result.stdout))
        if blast_record.alignments:
            return blast_record.alignments[0].hsps[0].score
    return 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Parallelize BLAST queries
def process_sequence(fasta):
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score = get_blast_score(sequence)
    return feature, kmer_feature, blast_score

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_sequence, merged_df['FASTA']))

for feature, kmer_feature, blast_score in results:
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Optionally, save the model for future use
joblib.dump(model, 'random_forest_model.pkl')

# Save the blast scores for future use
features_df.to_csv('blast_scores.csv', index=False)

# Clean up temporary files
os.remove('temp_query.fasta')

ValueError: More than one record found in handle

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures
import joblib
import threading

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Use a unique filename for each thread
    temp_fasta_filename = f'temp_query_{threading.get_ident()}.fasta'
    
    # Write the sequence to a temporary FASTA file
    with open('temp_query.fasta', 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', 'temp_query.fasta', '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    # Remove the temporary FASTA file
    os.remove(temp_fasta_filename)
    
    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_record = NCBIXML.read(io.StringIO(result.stdout))
        if blast_record.alignments:
            return blast_record.alignments[0].hsps[0].score
    return 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Parallelize BLAST queries
def process_sequence(fasta):
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score = get_blast_score(sequence)
    return feature, kmer_feature, blast_score

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_sequence, merged_df['FASTA']))

for feature, kmer_feature, blast_score in results:
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Optionally, save the model for future use
joblib.dump(model, 'random_forest_model.pkl')

# Save the blast scores for future use
features_df.to_csv('blast_scores.csv', index=False)

# Clean up temporary files
os.remove('temp_query.fasta')

FileNotFoundError: [WinError 2] Het systeem kan het opgegeven bestand niet vinden: 'temp_query_15960.fasta'

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures
import joblib
import threading

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Use a unique filename for each thread
    temp_fasta_filename = f'temp_query_{threading.get_ident()}.fasta'
    
    # Write the sequence to a temporary FASTA file
    with open('temp_query.fasta', 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', 'temp_query.fasta', '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    # Remove the temporary FASTA file
    os.remove(temp_fasta_filename)

    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_record = NCBIXML.read(io.StringIO(result.stdout))
        if blast_record.alignments:
            return blast_record.alignments[0].hsps[0].score
    return 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Optionally, save the model for future use
joblib.dump(model, 'random_forest_model.pkl')

# Save the blast scores for future use
features_df.to_csv('blast_scores.csv', index=False)

# Clean up temporary files
os.remove('temp_query.fasta')

FileNotFoundError: [WinError 2] Het systeem kan het opgegeven bestand niet vinden: 'temp_query_10076.fasta'

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import concurrent.futures
import joblib
import threading

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Use a unique filename for each thread
    temp_fasta_filename = f'temp_query_{threading.get_ident()}.fasta'
    
    # Write the sequence to a temporary FASTA file
    with open('temp_query.fasta', 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', 'temp_query.fasta', '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )

    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_record = NCBIXML.read(io.StringIO(result.stdout))
        if blast_record.alignments:
            return blast_record.alignments[0].hsps[0].score
    return 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Optionally, save the model for future use
joblib.dump(model, 'random_forest_model.pkl')

# Save the blast scores for future use
features_df.to_csv('blast_scores.csv', index=False)

# Clean up temporary files
os.remove('temp_query.fasta')

ValueError: More than one record found in handle

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import joblib
import threading

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Use a unique filename for each thread
    temp_fasta_filename = f'temp_query_{threading.get_ident()}.fasta'
    
    # Write the sequence to a temporary FASTA file
    with open(temp_fasta_filename, 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', temp_fasta_filename, '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
    # Remove the temporary FASTA file
    os.remove(temp_fasta_filename)
    
    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_records = NCBIXML.parse(io.StringIO(result.stdout))
        for blast_record in blast_records:
            if blast_record.alignments:
                return blast_record.alignments[0].hsps[0].score
    return 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Optionally, save the model for future use
joblib.dump(model, 'random_forest_model.pkl')

# Save the blast scores for future use
features_df.to_csv('blast_scores.csv', index=False)

Accuracy: 0.8975
Classification Report:
              precision    recall  f1-score   support

         abj       0.84      0.98      0.91       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.93      0.80      0.86       135
         stf       1.00      0.29      0.44         7
         stx       0.96      0.99      0.97        97

    accuracy                           0.90       400
   macro avg       0.53      0.44      0.45       400
weighted avg       0.89      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
features_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] # + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.68
Classification Report:
              precision    recall  f1-score   support

         abj       0.67      0.71      0.69       156
         asj       0.00      0.00      0.00         1
         hex       1.00      1.00      1.00         2
         ssj       1.00      0.50      0.67         2
         std       0.60      0.59      0.60       135
         stf       0.50      0.14      0.22         7
         stx       0.79      0.79      0.79        97

    accuracy                           0.68       400
   macro avg       0.65      0.53      0.57       400
weighted avg       0.68      0.68      0.68       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
features_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = [ 'gc_content', 'blast_score'] # + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.61
Classification Report:
              precision    recall  f1-score   support

         abj       0.61      0.66      0.63       156
         asj       0.00      0.00      0.00         1
         hex       1.00      1.00      1.00         2
         ssj       0.00      0.00      0.00         2
         std       0.53      0.53      0.53       135
         stf       1.00      0.14      0.25         7
         stx       0.74      0.69      0.72        97

    accuracy                           0.61       400
   macro avg       0.55      0.43      0.45       400
weighted avg       0.62      0.61      0.61       400



In [ ]:
df = pd.concat([merged_df, features_df, kmer_df], axis=1)
features_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'blast_score'] # + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.6325
Classification Report:
              precision    recall  f1-score   support

         abj       0.59      0.74      0.66       156
         asj       0.00      0.00      0.00         1
         hax       0.00      0.00      0.00         0
         hex       1.00      1.00      1.00         2
         ssj       1.00      1.00      1.00         2
         std       0.54      0.41      0.46       135
         stf       0.00      0.00      0.00         7
         stx       0.80      0.80      0.80        97

    accuracy                           0.63       400
   macro avg       0.49      0.49      0.49       400
weighted avg       0.62      0.63      0.62       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\sit

In [ ]:
feature_columns = ['length', 'gc_content'] # + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.5075
Classification Report:
              precision    recall  f1-score   support

         abj       0.54      0.56      0.55       156
         asj       0.00      0.00      0.00         1
         hex       1.00      1.00      1.00         2
         ssj       0.00      0.00      0.00         2
         std       0.44      0.43      0.43       135
         stf       0.17      0.14      0.15         7
         stx       0.56      0.57      0.56        97

    accuracy                           0.51       400
   macro avg       0.39      0.39      0.39       400
weighted avg       0.50      0.51      0.50       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
features_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score'] # + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.68
Classification Report:
              precision    recall  f1-score   support

         abj       0.67      0.71      0.69       156
         asj       0.00      0.00      0.00         1
         hex       1.00      1.00      1.00         2
         ssj       1.00      0.50      0.67         2
         std       0.60      0.59      0.60       135
         stf       0.50      0.14      0.22         7
         stx       0.79      0.79      0.79        97

    accuracy                           0.68       400
   macro avg       0.65      0.53      0.57       400
weighted avg       0.68      0.68      0.68       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)
features_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

KeyError: "['blast_score'] not in index"

In [ ]:
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)
features_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

KeyError: "['blast_score'] not in index"

In [ ]:
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)
df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

KeyError: '[\'>NC\', \'NC_\', \'C_0\', \'_03\', \'039\', \'399\', \'992\', \'929\', \'29.\', \'9.1\', \'.1:\', \'1:1\', \':16\', \'166\', \'668\', \'687\', \'87-\', \'7-1\', \'-19\', \'191\', \'911\', \'112\', \'12 \', \'2 A\', \' Ac\', \'Act\', \'cti\', \'tin\', \'ini\', \'nia\', \'ia \', \'a e\', \' eq\', \'equ\', \'qui\', \'uin\', \'ina\', \'na \', \'a m\', \' mi\', \'mit\', \'ito\', \'toc\', \'och\', \'cho\', \'hon\', \'ond\', \'ndr\', \'dri\', \'rio\', \'ion\', \'on,\', \'n, \', \', c\', \' co\', \'com\', \'omp\', \'mpl\', \'ple\', \'let\', \'ete\', \'te \', \'e g\', \' ge\', \'gen\', \'eno\', \'nom\', \'ome\', \'me\\n\', \'e\\nA\', \'\\nAT\', \'ATG\', \'TGA\', \'GAA\', \'AAT\', \'ATA\', \'TAA\', \'AAA\', \'ATT\', \'TTT\', \'TTA\', \'AAC\', \'ACT\', \'CTC\', \'TCG\', \'CGT\', \'GTT\', \'TTG\', \'GAG\', \'AGT\', \'GTA\', \'TAT\', \'TTC\', \'TCC\', \'CCA\', \'CAC\', \'CTA\', \'ATC\', \'TCA\', \'ACA\', \'CAA\', \'AAG\', \'AGA\', \'GAT\', \'CGG\', \'GGA\', \'CTT\', \'TA\\n\', \'A\\nT\

In [ ]:
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)
features_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

KeyError: "['blast_score'] not in index"

In [ ]:
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)
feature_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

KeyError: "['blast_score'] not in index"

In [ ]:
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)
features_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

KeyError: "['blast_score'] not in index"

In [ ]:
# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)

# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)
features_df= pd.read_csv("blast_scores.csv")
# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
features_columns = ['length', 'gc_content', 'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[features_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

KeyError: "['blast_score'] not in index"

In [ ]:
# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.8975
Classification Report:
              precision    recall  f1-score   support

         abj       0.84      0.98      0.91       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.93      0.80      0.86       135
         stf       1.00      0.29      0.44         7
         stx       0.96      0.99      0.97        97

    accuracy                           0.90       400
   macro avg       0.53      0.44      0.45       400
weighted avg       0.89      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9
Classification Report:
              precision    recall  f1-score   support

         abj       0.84      0.98      0.90       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.94      0.81      0.87       135
         stf       1.00      0.29      0.44         7
         stx       0.97      0.99      0.98        97

    accuracy                           0.90       400
   macro avg       0.54      0.44      0.46       400
weighted avg       0.90      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = [ 'gc_content', 'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.8925
Classification Report:
              precision    recall  f1-score   support

         abj       0.84      0.98      0.90       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.94      0.79      0.85       135
         stf       1.00      0.29      0.44         7
         stx       0.94      0.99      0.96        97

    accuracy                           0.89       400
   macro avg       0.53      0.43      0.45       400
weighted avg       0.89      0.89      0.88       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length',  'blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.8875
Classification Report:
              precision    recall  f1-score   support

         abj       0.83      0.98      0.90       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.94      0.77      0.85       135
         stf       1.00      0.29      0.44         7
         stx       0.93      0.99      0.96        97

    accuracy                           0.89       400
   macro avg       0.53      0.43      0.45       400
weighted avg       0.88      0.89      0.88       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['blast_score']  + list(kmer_df.columns)  # Add more features as needed
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9
Classification Report:
              precision    recall  f1-score   support

         abj       0.83      0.99      0.90       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.95      0.80      0.87       135
         stf       1.00      0.29      0.44         7
         stx       0.97      0.99      0.98        97

    accuracy                           0.90       400
   macro avg       0.54      0.44      0.46       400
weighted avg       0.90      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['blast_score']    # Add more features as needed length and gc doen ni zo veel
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.6675
Classification Report:
              precision    recall  f1-score   support

         abj       0.63      0.83      0.71       156
         asj       0.00      0.00      0.00         1
         hex       0.67      1.00      0.80         2
         ssj       0.00      0.00      0.00         2
         std       0.60      0.42      0.50       135
         stf       0.00      0.00      0.00         7
         stx       0.82      0.81      0.82        97

    accuracy                           0.67       400
   macro avg       0.39      0.44      0.40       400
weighted avg       0.65      0.67      0.65       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
#blast_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    # blast_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    # blast_scores.append(blast_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
#features_df['blast_score'] = blast_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = []  + list(kmer_df.columns)  # Add more features as needed length and gc doen ni zo veel
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9025
Classification Report:
              precision    recall  f1-score   support

         abj       0.84      0.99      0.91       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.95      0.81      0.87       135
         stf       1.00      0.29      0.44         7
         stx       0.97      0.99      0.98        97

    accuracy                           0.90       400
   macro avg       0.54      0.44      0.46       400
weighted avg       0.90      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import joblib
import threading

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Use a unique filename for each thread
    temp_fasta_filename = f'temp_query_{threading.get_ident()}.fasta'
    
    # Write the sequence to a temporary FASTA file
    with open(temp_fasta_filename, 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', temp_fasta_filename, '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
    # Remove the temporary FASTA file
    os.remove(temp_fasta_filename)
    
    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_records = NCBIXML.parse(io.StringIO(result.stdout))
        for blast_record in blast_records:
            if blast_record.alignments:
                hsp = blast_record.alignments[0].hsps[0]
                return hsp.score, hsp.bits
    return 0, 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []
bit_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score, bit_scores = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)
    bit_scores.append(bit_scores)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
# features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores
features_df['bit_score'] = bit_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score','bit_score']  + list(kmer_df.columns)  # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

AttributeError: 'float' object has no attribute 'append'

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import joblib
import threading

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Use a unique filename for each thread
    temp_fasta_filename = f'temp_query_{threading.get_ident()}.fasta'
    
    # Write the sequence to a temporary FASTA file
    with open(temp_fasta_filename, 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', temp_fasta_filename, '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
    # Remove the temporary FASTA file
    os.remove(temp_fasta_filename)
    
    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_records = NCBIXML.parse(io.StringIO(result.stdout))
        for blast_record in blast_records:
            if blast_record.alignments:
                hsp = blast_record.alignments[0].hsps[0]
                return hsp.score, hsp.bits
    return 0, 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []
bit_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score, bit_scores = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)
    bit_scores.append(bit_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
# features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores
features_df['bit_score'] = bit_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score','bit_score']  + list(kmer_df.columns)  # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

AttributeError: 'float' object has no attribute 'append'

In [ ]:
import pandas as pd
import subprocess
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import io
import joblib
import threading

# Step 1: Read the CSV file and write to a FASTA file
csv_file = 'AMP_species_list_COX1.csv'
df = pd.read_csv(csv_file)

with open('COX1_sequences.fasta', 'w') as fasta_file:
    for index, row in df.iterrows():
        fasta_file.write(f">{row['FASTA']}\n")

# Step 2: Create the BLAST database
subprocess.run(['makeblastdb', '-in', 'COX1_sequences.fasta', '-dbtype', 'nucl', '-out', 'COX1_blastdb'])

# Function to extract features from a sequence
def extract_features(sequence):
    features = {
        'length': len(sequence),
        'gc_content': (sequence.count('G') + sequence.count('C')) / len(sequence)
    }
    return features

# Function to extract k-mer counts from a sequence
def get_kmers(sequence, k=3):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return Counter(kmers)

# Function to perform local BLAST alignment and extract alignment score
def get_blast_score(sequence, db='COX1_blastdb'):
    # Use a unique filename for each thread
    temp_fasta_filename = f'temp_query_{threading.get_ident()}.fasta'
    
    # Write the sequence to a temporary FASTA file
    with open(temp_fasta_filename, 'w') as temp_fasta:
        temp_fasta.write(f'>query\n{sequence}\n')
    
    # Run the local BLAST search
    result = subprocess.run(
        ['blastn', '-query', temp_fasta_filename, '-db', db, '-outfmt', '5'],
        capture_output=True, text=True
    )
    
    # Remove the temporary FASTA file
    os.remove(temp_fasta_filename)
    
    # Check if the BLAST search produced valid output
    if result.stdout.strip():
        # Parse the BLAST XML output
        blast_records = NCBIXML.parse(io.StringIO(result.stdout))
        for blast_record in blast_records:
            if blast_record.alignments:
                hsp = blast_record.alignments[0].hsps[0]
                return hsp.score, hsp.bits
    return 0, 0

# Load the CSV files
sequences_df = pd.read_csv('gene_FASTA_COX1_final.csv')  # Contains sequence and ID
animals_df = pd.read_csv('gene_IDS_COX1_retrial.csv')  # Contains ID and animal information
characteristics_df = pd.read_csv('AMP_species_list.csv')  # Contains animal and characteristic

# Merge the DataFrames
merged_df = pd.merge(sequences_df, animals_df, on='gene_ID', how='inner')  # Merge on 'ID'
merged_df = pd.merge(merged_df, characteristics_df, on='ID', how='inner')  # Merge on 'Animal'
merged_df.to_csv('AMP_species_list_COX1.csv', index=False)

# Extract features from the FASTA sequences
features = []
kmer_features = []
blast_scores = []
bit_scores = []

# Process each sequence one by one
for fasta in merged_df['FASTA']:
    sequence = str(Seq(fasta))
    feature = extract_features(sequence)
    kmer_feature = get_kmers(sequence)
    blast_score, bit_score = get_blast_score(sequence)
    
    features.append(feature)
    kmer_features.append(kmer_feature)
    blast_scores.append(blast_score)
    bit_scores.append(bit_score)

# Convert features to a DataFrame
features_df = pd.DataFrame(features)
# features_df= pd.read_csv("blast_scores.csv")
# Convert k-mer features to a DataFrame
kmer_df = pd.DataFrame(kmer_features).fillna(0)

# Add BLAST scores to the features DataFrame
features_df['blast_score'] = blast_scores
features_df['bit_score'] = bit_scores

# Combine features with the merged DataFrame
df = pd.concat([merged_df, features_df, kmer_df], axis=1)

# Prepare the data
# Assuming 'Characteristic' is the column you want to predict
# Specify the columns you want to use for X
feature_columns = ['length', 'gc_content', 'blast_score','bit_score']  + list(kmer_df.columns)  # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9
Classification Report:
              precision    recall  f1-score   support

         abj       0.84      0.98      0.91       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.93      0.81      0.87       135
         stf       1.00      0.29      0.44         7
         stx       0.97      0.99      0.98        97

    accuracy                           0.90       400
   macro avg       0.53      0.44      0.46       400
weighted avg       0.89      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
features_df.to_csv('blast_scores.csv', index=False)

In [ ]:
feature_columns = ['blast_score','bit_score']  + list(kmer_df.columns)  # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.8975
Classification Report:
              precision    recall  f1-score   support

         abj       0.84      0.98      0.90       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.94      0.80      0.86       135
         stf       1.00      0.29      0.44         7
         stx       0.96      0.99      0.97        97

    accuracy                           0.90       400
   macro avg       0.53      0.44      0.46       400
weighted avg       0.89      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
feature_columns = ['bit_score']  + list(kmer_df.columns)  # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9
Classification Report:
              precision    recall  f1-score   support

         abj       0.83      0.99      0.90       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.95      0.80      0.87       135
         stf       1.00      0.29      0.44         7
         stx       0.97      0.99      0.98        97

    accuracy                           0.90       400
   macro avg       0.54      0.44      0.46       400
weighted avg       0.90      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
feature_columns = []  + list(kmer_df.columns)  # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9025
Classification Report:
              precision    recall  f1-score   support

         abj       0.84      0.99      0.91       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.95      0.81      0.87       135
         stf       1.00      0.29      0.44         7
         stx       0.97      0.99      0.98        97

    accuracy                           0.90       400
   macro avg       0.54      0.44      0.46       400
weighted avg       0.90      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
feature_columns = ['blast_score','bit_score']   # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.67
Classification Report:
              precision    recall  f1-score   support

         abj       0.63      0.83      0.72       156
         asj       0.00      0.00      0.00         1
         hex       0.67      1.00      0.80         2
         ssj       0.00      0.00      0.00         2
         std       0.61      0.42      0.50       135
         stf       0.00      0.00      0.00         7
         stx       0.82      0.81      0.82        97

    accuracy                           0.67       400
   macro avg       0.39      0.44      0.40       400
weighted avg       0.65      0.67      0.65       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
feature_columns = [,'bit_score']   # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score

SyntaxError: invalid syntax (<ipython-input-43-f5a1d6d03450>, line 1)

In [ ]:
feature_columns = [,'bit_score']   # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

SyntaxError: invalid syntax (<ipython-input-44-efed932c7aa0>, line 1)

In [ ]:
feature_columns = ['bit_score']   # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.67
Classification Report:
              precision    recall  f1-score   support

         abj       0.63      0.83      0.72       156
         asj       0.00      0.00      0.00         1
         hex       0.67      1.00      0.80         2
         ssj       0.00      0.00      0.00         2
         std       0.61      0.42      0.50       135
         stf       0.00      0.00      0.00         7
         stx       0.82      0.81      0.82        97

    accuracy                           0.67       400
   macro avg       0.39      0.44      0.40       400
weighted avg       0.65      0.67      0.65       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
feature_columns = []  + list(kmer_df.columns)  # Add more features as needed length and gc doen ni veel kmer wint zelf zonder blast score
X = df[feature_columns]
y = df['Mod']  # Replace with your target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=300, random_state=37)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9025
Classification Report:
              precision    recall  f1-score   support

         abj       0.84      0.99      0.91       156
         asj       0.00      0.00      0.00         1
         hex       0.00      0.00      0.00         2
         ssj       0.00      0.00      0.00         2
         std       0.95      0.81      0.87       135
         stf       1.00      0.29      0.44         7
         stx       0.97      0.99      0.98        97

    accuracy                           0.90       400
   macro avg       0.54      0.44      0.46       400
weighted avg       0.90      0.90      0.89       400



c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gebruiker\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
joblib.dump(model, 'random_forest_model_kmer.pkl')

['random_forest_model_kmer.pkl']